## Network Improvements (in development, skip for now)
---
Use this section to add new links or improve existing ones. New links should be drawn in a GIS program. Functions for adding reference columns and splitting existing links will be added in the future, but need to be manually added for now. If wanting to see the impacts of only one improvement/new link, be sure to export that as a separate network.

### Improving an existing link with a protected bike lane

In [ ]:
#extend protected bike lane on 10th st
improvements = links.copy()
changes = ['10th Street Northwest','10th Street Northeast']
improvements.loc[improvements['name'].isin(changes),'pbl'] = 1

#change a specific link back
improvements.loc[improvements['A_B']=='9543446970_69614574','pbl'] = 0
improvements.loc[improvements['A_B']=='69614574_9543446970','pbl'] = 0

#export the new improvement for mapping/visualizations
tenth_st  = improvements[improvements['pbl'] != links['pbl']].copy()
tenth_st.geometry = tenth_st.buffer(300)
tenth_st = tenth_st.dissolve()
tenth_st.to_file(export_fp/'network_improvements.gpkg',layer='New Protected Bike Lane')

#create a copy so the improvement from the protected bike lane can be tested separately
just_pbl = improvements.copy()

### Bring in feature(s) drawn in GIS and add to network

In [ ]:
#bring in new feature and add to old links
new = gpd.read_file(export_fp/'network_improvements.gpkg',layer='morningside_path')
morningside_path = new.copy()
morningside_path.geometry = morningside_path.buffer(300)
morningside_path = morningside_path.dissolve()
morningside_path.to_file(export_fp / "network_improvements.gpkg", layer = 'New Multi-Use Path')

#add ref ids ONLY if no new nodes are added
new = add_ref_ids(new,nodes)

#make reverse links if they are two way features
new = create_reverse_links(new)
new['dist'] = new.length
new['mins'] = new['dist'] / 5280 / 8 * 60

#add to network with 10st pbl
improvements = pd.concat([improvements,new],ignore_index=True)

#just add path
just_path = links.copy()
just_path = pd.concat([just_path,new],ignore_index=True)

c:\Users\tpassmore6\Anaconda3\envs\geo-env\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\tpassmore6\Anaconda3\envs\geo-env\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


Reference IDs successfully added to links.
